In [1]:
!pip install pynytimes

     |████████████████████████████████| 71kB 3.7MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.24.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [3]:
import pandas as pd
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import matplotlib.pyplot as plt
import tweepy
from pynytimes import NYTAPI
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('drive/My Drive/')

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
#initialize setniment intensity analyzer 
nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [6]:
#initialize nyt api session
api_key = 'H7r2cyZrHFQslp5pT5CCcRbikNrBONNd'
nyt = NYTAPI(api_key)

In [19]:
#create pandas df
df = pd.DataFrame(columns = ['publishdate', 'topic', 'url', 'title', 'description',  'sentiment score']) 

In [20]:
#initialize params for article search
topics = ['covid','race','jobs','education','trump']
start, end = '2020-01-22','2020-11-17'

In [21]:
#function that get maximum distinct description of text sample
def get_description(art):
    lead,snippet = art['lead_paragraph'],art['snippet']
    encoding = {0:lead,1:snippet}
    #if they are the same or one is apart of the other get max sample
    if encoding[np.argmin([len(lead),len(snippet)])] in encoding[np.argmax([len(lead),len(snippet)])]:
        description = encoding[np.argmax([len(lead),len(snippet)])]
    #else combine both for a larger text sample
    else:
        description = snippet+" " +lead
    return(description)

In [22]:
#iterate through each day in range provided 
index = 0
for day in pd.date_range(start=start, end=end).strftime('%Y-%m-%d'):
    print(day)
    print("========================")
    #iterate through each topic provided
    for topic in topics: 
        print(topic,day)
        comp_scores = []
        #search for top 10 most relevant articles of current topic
        articles = nyt.article_search(query = topic,results = 10,
                                          dates = {
                                                    "begin": pd.to_datetime(day),
                                                    "end": pd.to_datetime(day)
                                                    },
                                            options = {
                                                    "sort": "relevance",
                                                    "sources": ["New York Times"],
                                                    'document_type':'article'
                                                    }
                                        )
        #get description and sentiment score for each article
        for this_article in articles:
            # print(f"{this_article['headline']['main']}")
            description = get_description(this_article)
            comp_score = sid.polarity_scores(description)['compound']
            df.loc[index] = [day,topic,this_article['web_url'],this_article['headline']['main'],description,comp_score]
            index += 1
            comp_scores.append(comp_score)

        #display mean sentiment score for current topic on current day
        print(f'mean score: {round(np.mean(comp_scores),3)}')
        print('==========')
    print("\n")

2020-01-22
covid 2020-01-22
mean score: -0.618
race 2020-01-22
mean score: 0.312
jobs 2020-01-22
mean score: -0.167
education 2020-01-22
mean score: 0.201
trump 2020-01-22
mean score: 0.03


2020-01-23
covid 2020-01-23
mean score: -0.259
race 2020-01-23
mean score: 0.034
jobs 2020-01-23


KeyboardInterrupt: ignored

In [14]:
df.sample(n=30)

,Unnamed: 0,publishdate,topic,url,title,description,sentiment score
8127,8127,2020-07-10,race,https://www.nytimes.com/2020/07/10/world/canad...,The Man Striving to Be the ‘Canadian Obama’,"Balarama Holness, 36, a law student and commun...",-0.4574
3320,3320,2020-03-31,education,https://www.nytimes.com/2020/03/31/magazine/ed...,‘I Always Write With a Sense of Shame’,"How Édouard Louis, a working-class gay man fro...",-0.3182
3527,3527,2020-04-05,covid,https://www.nytimes.com/2020/04/05/opinion/cor...,I’m a Doctor at the ‘Epicenter of the Epicenter’,Let’s not forget the social failures that allo...,-0.1366
13986,13986,2020-11-07,jobs,https://www.nytimes.com/2020/11/07/us/politics...,Biden defeats Trump in Nevada.,Joseph R. Biden Jr. won the state of Nevada on...,0.2732
6381,6381,2020-06-04,covid,https://www.nytimes.com/2020/06/04/smarter-liv...,The Pandemic Is Stressing Your Body in New Ways,"Tingling feet, digestive troubles, skin issues...",0.3049
5214,5214,2020-05-10,jobs,https://www.nytimes.com/2020/05/10/business/ai...,The Airline Business Is Terrible. It Will Prob...,An industry that is intimately familiar with f...,0.1263
8543,8543,2020-07-18,trump,https://www.nytimes.com/2020/07/17/us/politics...,F.B.I. Agent in Russia Inquiry Saw Basis in Ea...,Newly declassified documents added more fodder...,0.3400
9867,9867,2020-08-15,covid,https://www.nytimes.com/2020/08/15/us/covid-co...,"As Colleges Move Classes Online, Families Rebe...",Schools face rising demands for tuition rebate...,-0.3182
1270,1270,2020-02-18,jobs,https://www.nytimes.com/2020/02/18/books/revie...,Her Blog Post About Uber Upended Big Tech. Now...,"In “Whistleblower,” Susan Fowler, a former sof...",-0.4522
14123,14123,2020-11-10,race,https://www.nytimes.com/article/what-races-hav...,The Races That Haven’t Been Called,Because so many mail ballots still need to be ...,0.5719


In [13]:
#save df as csv file
df.to_csv(os.path.join(os.getcwd(),'nyt_2020.csv'))